In [0]:
from cryptography.fernet import Fernet
from pyspark.sql.functions import lit, udf, md5
from pyspark.sql.types import StringType
import pickle

## Definição de Funções

In [0]:
%python
def retira_acento(palavra):
  cDe   = "áéíóúàèìòùâêîôûãõäëïöüçñÿýÁÉÍÓÚÀÈÌÒÙÂÊÎÔÛÃÕÄËÏÖÜÇÑŸÝ"
  cPara = "aeiouaeiouaeiouaoaeioucnyyAEIOUAEIOUAEIOUAOAEIOUCNYY"
  cReturn = ""
  count = 0

  try:
    while count < len(palavra):
      letra = palavra[count]
      indice = cDe.find(letra)
      count += 1
      if indice == -1:
        cReturn += letra
      else:
        cReturn += cPara[indice]
  except:
    cReturn = ""
    
  return cReturn

spark.udf.register("retira_acento", retira_acento)

Out[7]: <function __main__.retira_acento(palavra)>

In [0]:
%python
def encrypt_val(clear_text,MASTER_KEY):
    from cryptography.fernet import Fernet
    f = Fernet(MASTER_KEY)
    clear_text_b=bytes(clear_text, 'utf-8')
    cipher_text = f.encrypt(clear_text_b)
    cipher_text = str(cipher_text.decode('ascii'))
    return cipher_text
# Register UDF's
fct_encrypt = udf(encrypt_val, StringType())
spark.udf.register('encrypt_val',encrypt_val)

Out[8]: <function __main__.encrypt_val(clear_text, MASTER_KEY)>

In [0]:
%python
def decrypt_val(cipher_text,MASTER_KEY):
    from cryptography.fernet import Fernet
    f = Fernet(MASTER_KEY)
    clear_val=f.decrypt(cipher_text.encode()).decode()
    return clear_val
# Register UDF's
fct_decrypt = udf(decrypt_val, StringType())
spark.udf.register('decrypt_val',decrypt_val)

Out[9]: <function __main__.decrypt_val(cipher_text, MASTER_KEY)>

## Recebimento de parâmetros de entrada

In [0]:
%python
# Criar parâmetro no Data Factory com o nome par_ADF_nm_arq_metadados
try:
  par_ADF_nm_arq_metadados = dbutils.widgets.get("par_ADF_nm_arq_metadados")
  print('# Execução via ADF')
  print('#')
except:
  # para rodar direto pelo Databricks, é necessário preencher o parâmetro abaixo.
  #par_ADF_nm_arq_metadados = 'metadados_mascaramento.csv'
  par_ADF_nm_arq_metadados = ''
  print('# Erro ou execução fora do ADF')
  print('#')
  
print('# par_ADF_nm_arq_metadados => ' + str(par_ADF_nm_arq_metadados))
print('#')

if par_ADF_nm_arq_metadados is None:
  print('# Parametro par_ADF_nm_arq_metadados obrigatorio.')
  dbutils.notebook.exit(1)

# Erro ou execução fora do ADF
#
# par_ADF_nm_arq_metadados => metadados_mascaramento.csv
#

In [0]:
%python
# Criar parâmetro no Data Factory com o nome par_ADF_nm_arq_dados
try:
  par_ADF_nm_arq_dados = dbutils.widgets.get("par_ADF_nm_arq_dados")
  print('# Execução via ADF')
  print('#')
except:
  # para rodar direto pelo Databricks, é necessário preencher o parâmetro abaixo.
  #par_ADF_nm_arq_dados = 'ANS_REGIME_INTERNACAO.csv'
  par_ADF_nm_arq_dados = ''
  print('# Erro ou execução fora do ADF')
  print('#')
  
print('# par_ADF_nm_arq_dados => ' + str(par_ADF_nm_arq_dados))
print('#')

if par_ADF_nm_arq_dados is None:
  print('# Parametro par_ADF_nm_arq_dados obrigatorio.')
  dbutils.notebook.exit(1)

# Erro ou execução fora do ADF
#
# par_ADF_nm_arq_dados => ANS_REGIME_INTERNACAO.csv
#

## Leitura dos Arquivos

In [0]:
%python
path_e_arquivo_meta = 'dbfs:/mnt/UHG/metadados/' + par_ADF_nm_arq_metadados
print('# Lendo ' + path_e_arquivo_meta)

df_metadado_readed = spark.read.format('csv')\
               .option("Header","True")\
               .option("inferSchema","False")\
               .option("delimiter",";")\
               .load(path_e_arquivo_meta)
print('#')
print('# Qtd regs lidos => ' + str(df_metadado_readed.count()))

# Lendo dbfs:/mnt/UHG/metadados/metadados_mascaramento.csv
#
# Qtd regs lidos => 1895

In [0]:
%python
path_e_arquivo = 'dbfs:/mnt/UHG/raw/sisamil/2021/8/2/' + par_ADF_nm_arq_dados
print('# Lendo ' + path_e_arquivo)

df_origem_readed = spark.read.format('csv')\
               .option("Header","True")\
               .option("inferSchema","False")\
               .option("delimiter",";")\
               .load(path_e_arquivo)
print('#')
print('# Qtd regs lidos => ' + str(df_origem_readed.count()))

# Lendo dbfs:/mnt/UHG/raw/sisamil/2021/8/2/ANS_REGIME_INTERNACAO.csv
#
# Qtd regs lidos => 260

### Ajustando nome das colunas

In [0]:
%python
print('# Ajustando nome das colunas do df de metadados.')
df2 = df_metadado_readed

for campo in df_metadado_readed.columns:
  novo_campo = retira_acento(campo.upper().replace(' ','_'))
  df2 = df2.withColumnRenamed(campo, novo_campo)

df2.createOrReplaceTempView('tpv_df2')
df2.printSchema()

# Ajustando nome das colunas do df de metadados.
root
-- TABELA_DE_ORIGEM: string (nullable = true)
-- COLUNA_DE_ORIGEM: string (nullable = true)
-- TIPO_DA_COLUNA_DE_ORIGEM: string (nullable = true)
-- CAMPO_MASCARADO: string (nullable = true)

In [0]:
%python
print('# Ajustando nome das colunas do df com dados.')
df3 = df_origem_readed

for campo in df_origem_readed.columns:
  novo_campo = retira_acento(campo.upper().replace(' ','_'))
  df3 = df3.withColumnRenamed(campo, novo_campo)

df3.createOrReplaceTempView('tpv_df3')
df3.printSchema()

# Ajustando nome das colunas do df com dados.
root
-- COD_TIPO_REGIME: string (nullable = true)
-- NOM_TIPO_REGIME: string (nullable = true)
-- COD_TERMO_TISS: string (nullable = true)
-- DT_INI_VIGENCIA_TISS: string (nullable = true)
-- DT_FIM_VIGENCIA_TISS: string (nullable = true)
-- DT_FIM_IMPLANTACAO_TISS: string (nullable = true)

## Gera chave de criptografia

In [0]:
# key generation
key = Fernet.generate_key()
print('# Chave gerada!')
print(key)

# Chave gerada!
b'CnfqiEUXmviP7lCIXmj_aLS6gT028RccOJGSSdFRKDw='

## Padroniza dados dos dataframes
    Retirada de Acentos e outros caracteres
    Padronização de letras (UPPERCASE)

In [0]:
%python
df_metadado = df2.selectExpr(['retira_acento(UPPER(' + campo + ')) AS ' + campo for campo in df2.columns])

In [0]:
df_metadado.show()

+--------------------+--------------------+------------------------+---------------+
 TABELA_DE_ORIGEM| COLUNA_DE_ORIGEM|TIPO_DA_COLUNA_DE_ORIGEM|CAMPO_MASCARADO|
+--------------------+--------------------+------------------------+---------------+
ANS_REGIME_INTERN...| COD_TIPO_REGIME| NUMBER| SIM|
ANS_REGIME_INTERN...| NOM_TIPO_REGIME| VARCHAR2| SIM|
ANS_REGIME_INTERN...| COD_TERMO_TISS| VARCHAR2| SIM|
ANS_REGIME_INTERN...|DT_INI_VIGENCIA_TISS| DATE| SIM|
ANS_REGIME_INTERN...|DT_FIM_VIGENCIA_TISS| DATE| SIM|
ANS_REGIME_INTERN...|DT_FIM_IMPLANTACA...| DATE| SIM|
 AUTORIZACAO| NUM_PEDIDO| VARCHAR2| SIM|
 AUTORIZACAO| COD_MOTIVO| VARCHAR2| SIM|
 AUTORIZACAO| COD_ACOMODACAO| NUMBER| SIM|
 AUTORIZACAO| COD_CID_SECUNDARIO| VARCHAR2| SIM|
 AUTORIZACAO| COD_TRATAMENTO| NUMBER| SIM|
 AUTORIZACAO| COD_CID_PRINCIPAL| VARCHAR2| SIM|
 AUTORIZACAO|DATA_PROVAVEL_INT...| DATE| SIM|
 AUTORIZACAO| NUM_DIARIAS| NUMBER| SIM|
 AUTORIZACAO| COD_INTERNACAO| VARCHAR2| SIM|
 AUTORIZACAO| NUM_DIARIAS_PRORROG| NUMBER| SIM|
 AUTORIZACAO| COD_MEIO_AUT| NUMBER| SIM|
 AUTORIZACAO| COD_TIPO_REMOCAO| NUMBER| SIM|
 AUTORIZACAO| COD_TS_CONTRATO| VARCHAR2| SIM|
 AUTORIZACAO| IND_CARATER| VARCHAR2| SIM|
+--------------------+--------------------+------------------------+---------------+
only showing top 20 rows

In [0]:
%python
df_dados = df3.selectExpr(['retira_acento(UPPER(' + campo + ')) AS ' + campo for campo in df3.columns])

In [0]:
%python
df_dados.show()

+---------------+---------------+--------------+--------------------+--------------------+-----------------------+
COD_TIPO_REGIME|NOM_TIPO_REGIME|COD_TERMO_TISS|DT_INI_VIGENCIA_TISS|DT_FIM_VIGENCIA_TISS|DT_FIM_IMPLANTACAO_TISS|
+---------------+---------------+--------------+--------------------+--------------------+-----------------------+
 1| NOME 1| 100| 01/08/2021| 20/03/2021| 03/05/2021|
 2| NOME 2| 102| 02/08/2021| 21/03/2021| 04/05/2021|
 3| NOME 3| 104| 03/08/2021| 22/03/2021| 05/05/2021|
 4| NOME 4| 106| 04/08/2021| 23/03/2021| 06/05/2021|
 5| NOME 5| 108| 05/08/2021| 24/03/2021| 07/05/2021|
 6| NOME 6| 110| 06/08/2021| 25/03/2021| 08/05/2021|
 7| NOME 7| 112| 07/08/2021| 26/03/2021| 09/05/2021|
 8| NOME 8| 114| 08/08/2021| 27/03/2021| 10/05/2021|
 9| NOME 9| 116| 09/08/2021| 28/03/2021| 11/05/2021|
 10| NOME 10| 118| 10/08/2021| 29/03/2021| 12/05/2021|
 11| NOME 11| 120| 11/08/2021| 30/03/2021| 13/05/2021|
 12| NOME 12| 122| 12/08/2021| 31/03/2021| 14/05/2021|
 13| NOME 13| 124| 13/08/2021| 01/04/2021| 15/05/2021|
 14| NOME 14| 126| 14/08/2021| 02/04/2021| 16/05/2021|
 15| NOME 15| 128| 15/08/2021| 03/04/2021| 17/05/2021|
 16| NOME 16| 130| 16/08/2021| 04/04/2021| 18/05/2021|
 17| NOME 17| 132| 17/08/2021| 05/04/2021| 19/05/2021|
 18| NOME 18| 134| 18/08/2021| 06/04/2021| 20/05/2021|
 19| NOME 19| 136| 19/08/2021| 07/04/2021| 21/05/2021|
 20| NOME 20| 138| 20/08/2021| 08/04/2021| 22/05/2021|
+---------------+---------------+--------------+--------------------+--------------------+-----------------------+
only showing top 20 rows

In [0]:
df_metadado.printSchema()
df_dados.printSchema()

root
-- TABELA_DE_ORIGEM: string (nullable = true)
-- COLUNA_DE_ORIGEM: string (nullable = true)
-- TIPO_DA_COLUNA_DE_ORIGEM: string (nullable = true)
-- CAMPO_MASCARADO: string (nullable = true)

root
-- COD_TIPO_REGIME: string (nullable = true)
-- NOM_TIPO_REGIME: string (nullable = true)
-- COD_TERMO_TISS: string (nullable = true)
-- DT_INI_VIGENCIA_TISS: string (nullable = true)
-- DT_FIM_VIGENCIA_TISS: string (nullable = true)
-- DT_FIM_IMPLANTACAO_TISS: string (nullable = true)

## Verifica se arquivo de dados tem metadados cadastrado

In [0]:
%python
# se nome do arquivo esta contido no metadados, proximo passo,senão, mensagem warning e para o processamento
# pega somente o nome do arquivo, sem a extensão de 3 posicoes (.csv, .xls)
var_nome_arquivo = par_ADF_nm_arq_dados[:-4]

print('# var_nome_arquivo => ' + var_nome_arquivo)
print('#')

var_qtd = df_metadado.select(df_metadado['TABELA_DE_ORIGEM'])\
           .filter(df_metadado['TABELA_DE_ORIGEM'] == var_nome_arquivo)\
           .count()
print('# Qtd metadados encontrados => ' + str(var_qtd))
print('#')

if int(var_qtd) == 0:
  print('# Nao encontrado metadados para o arquivo informado.')
  print('# Incluia os metadados ou informe outro arquivo.')
  print('###### PROCESSAMENTO INTERROMPIDO! #####')
  dbutils.notebook.exit(1)
else:
  print('# Metadados encontrado. Iniciando processo de criptografia.')

# var_nome_arquivo => ANS_REGIME_INTERNACAO
#
# Qtd metadados encontrados => 6
#
# Metadados encontrado. Iniciando processo de criptografia.

## Processo de Criptografia

### Seleciona lista de campos a criptografar

In [0]:
%python
lst_para_criptografar = df_metadado.select(df_metadado['COLUNA_DE_ORIGEM'])\
           .filter((df_metadado['CAMPO_MASCARADO'] == "SIM") & \
                   (df_metadado['TABELA_DE_ORIGEM'] == var_nome_arquivo)).rdd.flatMap(lambda x: x).collect()

In [0]:
%python
print('# Campos a criptografar:')
print('#----------------------#')
lst_para_criptografar

# Campos a criptografar:
#----------------------#
Out[24]: ['COD_TIPO_REGIME',
 'NOM_TIPO_REGIME',
 'COD_TERMO_TISS',
 'DT_INI_VIGENCIA_TISS',
 'DT_FIM_VIGENCIA_TISS',
 'DT_FIM_IMPLANTACAO_TISS']

### Define layout final do arquivo

In [0]:
%python
# carregando layout do dataframe final
# Campos que nao precisem de criptorafia, nao serao alterados
df_dados_final = df_dados

In [0]:
df_dados_final.printSchema()

root
-- COD_TIPO_REGIME: string (nullable = true)
-- NOM_TIPO_REGIME: string (nullable = true)
-- COD_TERMO_TISS: string (nullable = true)
-- DT_INI_VIGENCIA_TISS: string (nullable = true)
-- DT_FIM_VIGENCIA_TISS: string (nullable = true)
-- DT_FIM_IMPLANTACAO_TISS: string (nullable = true)

### Criptografa os campos da lista

In [0]:
%python
# para cada coluna do dataframe de dados, sera verificado se a mesma exista na lista de campos a criptografar.
# SE EXISTIR aplica criptografia, SENAO somente mantem a coluna
for nome_coluna in df_dados.columns:
  if nome_coluna in lst_para_criptografar:
    print('coluna crip -> ' + str(nome_coluna))
    df_dados_final = df_dados_final.withColumn(nome_coluna,(fct_encrypt(nome_coluna, lit(key))))
  else:
    print('coluna nao crip -> ' + str(nome_coluna))

coluna crip -> COD_TIPO_REGIME
coluna crip -> NOM_TIPO_REGIME
coluna crip -> COD_TERMO_TISS
coluna crip -> DT_INI_VIGENCIA_TISS
coluna crip -> DT_FIM_VIGENCIA_TISS
coluna crip -> DT_FIM_IMPLANTACAO_TISS

In [0]:
key

Out[28]: b'CnfqiEUXmviP7lCIXmj_aLS6gT028RccOJGSSdFRKDw='

### Para descodificar

In [0]:
%python
# É só remover os comentários abaixo
#
#for nome_coluna in df_dados_final2.columns:
#  if nome_coluna in lst_para_criptografar:
#    print('coluna crip -> ' + str(nome_coluna))
#    df_dados_final2 = df_dados_final2.withColumn(nome_coluna,(fct_decrypt(nome_coluna, lit(key))))


### Grava tabela criptografada e chave utilizada

In [0]:
%python
#gravar arquivo mascarado tabela_YYYYMMDD.parquet e outro arquivo com a chave como tabela_YYYYMMDD.key na pasta curation
df_dados_final.createOrReplaceTempView('tpv_dados_final')

nome_tabela = spark.sql('''select concat("tabela_",date_format(current_date(),"yyyyMMdd")) as nome_tabela''').collect()[0][0]
print('# nome_tabela => ' + nome_tabela)

path_location = '/mnt/UHG/curation/' + nome_tabela
print('# Salvando os dados em tabela.')
print('#')

df_table = spark.sql('''show tables in default like "{nome_tabela}" '''.format(nome_tabela=nome_tabela))

if str(df_table.count()) > '0':
  spark.sql('''DROP TABLE if exists {nome_tabela} '''.format(nome_tabela=nome_tabela))
  dbutils.fs.rm('{path_location}'.format(path_location=path_location),True)
  print('# Tabela ' + nome_tabela + ' excluída!')
  print('#')

df_dados_final.write.format('delta')\
              .mode('overwrite')\
              .option('overwriteSchema','true')\
              .saveAsTable('{nome_tabela}'.format(nome_tabela = nome_tabela)
                           ,overwrite=True
                           ,path='{path_location}'.format(path_location = path_location))
#spark.sql('''
#CREATE TABLE if not exists {nome_tabela}
#USING delta
#LOCATION "{path_location}"
#AS
#SELECT * FROM tpv_dados_final
#'''.format(nome_tabela = nome_tabela
#          ,path_location = path_location))
print('# Dados Salvos na tabela ' + nome_tabela)

# nome_tabela => tabela_20210803
# Salvando os dados em tabela.
#
# Tabela tabela_20210803 excluída!
#
# Dados Salvos na tabela tabela_20210803

In [0]:
%python
spark.sql('''desc detail  {nome_tabela}'''.format(nome_tabela = nome_tabela)).toPandas()

/databricks/spark/python/pyspark/sql/pandas/conversion.py:92: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
 Unable to convert the field properties. If this column is not necessary, you may consider dropping it or converting to primitive type before the conversion.
Direct cause: MapType is only supported with pyarrow 2.0.0 and above
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
 warnings.warn(msg)
Out[31]:

,format,id,name,description,location,createdAt,lastModified,partitionColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion
0,delta,1b944a9b-9292-4cd3-ad26-2c04255d9e74,default.tabela_20210803,None,dbfs:/mnt/UHG/curation/tabela_20210803,2021-08-03 03:22:44.095,2021-08-03 03:22:48,[],1,146068,{},1,2


In [0]:
dict_key = {'key':key}

In [0]:
dict_key

Out[33]: {'key': b'CnfqiEUXmviP7lCIXmj_aLS6gT028RccOJGSSdFRKDw='}

In [0]:
%python
path_key = '/dbfs' + path_location + '.key'
print('# Salvando chave de criptografia em ' + path_key)
print('#')

arquivo_chave = open(path_key,'wb')
pickle.dump(dict_key,arquivo_chave)
arquivo_chave.close()
print('# Chave gravada!')

# Salvando chave de criptografia em /dbfs/mnt/UHG/curation/tabela_20210803.key
#
# Chave gravada!

In [0]:
%python
print('# Leitura da chave para validacao.')
print('#')
lendo_chave = open(path_key,'rb')
output = pickle.load(lendo_chave,encoding='utf-8')
print('# chave lida -> ' + str(output))

# Leitura da chave para validacao.
#
# chave lida -> {'key': b'CnfqiEUXmviP7lCIXmj_aLS6gT028RccOJGSSdFRKDw='}